#### Computing the VSTOXX Index

* Gather data
* Compute sub-indexes
* Compute VSTOXX index

* VSTOXX is based on two sub-indexes
    * measures implied vol of imaginary options series with fixed time to expiry of 30 days
    * Achieved through linear interpolation of two nearest sub-indexes
    * VSTOXX 1M and VSTOXX 2M
    * On two days before expiry, VSTOXX 2M and VSTOXX 3M are used and extrapolated

* Gather data for option series up to 3 months of expiry

In [1]:
#
# Module with helper functions for the VSTOXX index calculation
#
# (c) The Python Quants GmbH
# For illustration purposes only.
# August 2014
#

TYEAR = 365 * 24 * 60 * 60.  # seconds of a standard year

import datetime as dt


def third_friday(date):
    ''' Returns the third friday of the month given by the datetime object date
    This is the day options expiry on.

    date: datetime object
        date of month for which third Friday is to be found
    '''
    
    number_days = date.day
    first_day = date - dt.timedelta(number_days - 1)
      # Reduce the given date to the first of the month.
      # Year and month stay the same.
    week_day = first_day.weekday()
      # What weekday is the first of the month (Mon=0, Tue=1, ...)
    day_delta = 4 - week_day  # distance to the next Friday 
    if day_delta < 0:
        day_delta += 7
    third_friday = first_day + dt.timedelta(day_delta + 14)
      # add that distance plus two weeks to the first of month
    return third_friday


def first_settlement_day(date):
    ''' Returns the next settlement date (third Friday of a month) following
    the date date.

    date: datetime object
        date for which following third Friday is to be found
    '''

    settlement_day_in_month = third_friday(date)
      # settlement date in the given month

    delta = (settlement_day_in_month - date).days
      # where are we relative to the settlement date in that month?
    
    if delta > 1:  # more than 1 day before ?
        return settlement_day_in_month
         # yes: take the settlement dates of this and the next month
    else:
        next_month = settlement_day_in_month + dt.timedelta(20)
          # no: shift the date of next month into the next month but one and ...
        settlement_day_next_month = third_friday(next_month)
          # ... compute that settlement day
        return settlement_day_next_month


def second_settlement_day(date):
    ''' Returns the second settlement date (third Friday of a month) following
    the date date.

    date: datetime object
        date for which second third Friday is to be found
    '''

    settlement_day_in_month = first_settlement_day(date)
      # settlement date in the given month
    next_month = settlement_day_in_month + dt.timedelta(20)
      # shift date to the next month
    return third_friday(next_month)  # settlement date of that month


def not_a_day_before_expiry(date):
    ''' Returns True if the date is NOT one day before or equal the third
    Friday in month

    date: datetime object
        date for which second third Friday is to be found
    '''

    settlement_day_in_month = third_friday(date)
    delta = (settlement_day_in_month - date).days
    if delta == 1 or delta == 0:
        return False
    else:
        return True

        
def compute_delta(date, settlement_day):
    ''' Computes the time (in seconds) from date 0:00 to the first settlement
    date 8:30 AM

    date: datetime object
        starting date
    settlement_day: datetime object
        relevant settlement day
    '''
   
    dummy_time_1 = dt.timedelta(seconds=43200)
      # seconds from midnight to 12:00
    dummy_time_2 = dt.timedelta(seconds=23400)
      # seconds from 17:30 to midnight
    settlement_date = settlement_day + dummy_time_1 + dummy_time_2
    delta_T_dummy = settlement_date - date
    delta_T = ((delta_T_dummy.days - 1) * 24 * 60 * 60 + 
                delta_T_dummy.seconds) / TYEAR
    return delta_T

                    


In [5]:
#
# Module to collect option series data
# from the web
# Source: www.eurexchange.com
# Data is needed to calculate the VSTOXX
# and its sub-indexes
#
# (c) Dr. Yves J. Hilpisch
# Listed Volatility and Variance Derivatives
#
import requests
import datetime as dt
import pandas as pd
import numpy as np
from io import StringIO
# from index_date_functions import *

#
# The URL template
#
url1 = 'http://www.eurexchange.com/action/exchange-en/'
url2 = '180106-180102/180102/onlineStats.do?productGroupId=846'
url3 = '&productId=19068&viewType=3&cp=%s&month=%s&year=%s&busDate=%s'
URL = url1 + url2 + url3


def collect_option_series(month, year, start):
    ''' Collects daily option data from web source.

    Parameters
    ==========
    month: int
        maturity month
    year: int
        maturity year
    start: datetime object
        starting date

    Returns
    =======
    dataset: pandas DataFrame object
        object containing the collected data
    '''
    end = dt.datetime.today()
    delta = (end - start).days

    dataset = pd.DataFrame()
    for t in range(0, delta):  # runs from start to today
        date = start + dt.timedelta(t)
        dummy = get_data(month, year, date)  # get data for one day
        if len(dummy) != 0:
            if len(dataset) == 0:
                dataset = dummy
            else:
                dataset = pd.concat((dataset, dummy))  # add data

    return dataset


def get_data(month, year, date):
    ''' Get the data for an option series.

    Parameters
    ==========
    month: int
        maturity month
    year: int
        maturity year
    date: datetime object
        the date for which the data is collected

    Returns
    =======
    dataset: pandas DataFrame object
        object containing call & put option data
    '''

    date_string = date.strftime('%Y%m%d')
    # loads the call data from the web
    data = get_data_from_www('Call', month, year, date_string)
    calls = parse_data(data, date)  # parse the raw data
    calls = calls.rename(columns={'Daily settlem. price': 'Call_Price'})

    calls = pd.DataFrame(calls.pop('Call_Price').astype(float))
    # the same for puts
    data = get_data_from_www('Put', month, year, date_string)
    puts = parse_data(data, date)
    puts = puts.rename(columns={'Daily settlem. price': 'Put_Price'})
    puts = pd.DataFrame(puts.pop('Put_Price').astype(float))

    dataset = merge_and_filter(puts, calls)   # merges the two time series

    return dataset


def get_data_from_www(oType, matMonth, matYear, date):
    ''' Retrieves the raw data of an option series from the web.

    Parameters
    ==========
    oType: string
        either 'Put' or 'Call'
    matMonth: int
        maturity month
    matYear: int
        maturity year
    date: string
        expiry in the format 'YYYYMMDD'

    Returns
    =======
    a: string
        raw text with option data
    '''

    url = URL % (oType, matMonth, matYear, date)  # parametrizes the URL
    a = requests.get(url).text
    return a


def merge_and_filter(puts, calls):
    ''' Gets two pandas time series for the puts and calls
    (from the same option series), merges them, filters out
    all options with prices smaller than 0.5 and
    returns the resulting DataFrame object.

    Parameters
    ==========
    puts: pandas DataFrame object
        put option data
    calls: pandas DataFrame object
        call option data

    Returns
    =======
    df: pandas DataFrame object
        merged & filtered options data
    '''

    df = calls.join(puts, how='inner')  # merges the two time series
    # filters all prices which are too small
    df = df[(df.Put_Price >= 0.5) & (df.Call_Price >= 0.5)]

    return df


def parse_data(data, date):
    ''' Parses the HTML table and transforms it into a CSV compatible
    format. The result can be directly imported into a pandas DataFrame.

    Parameters
    ==========
    data: string
        document containing the Web content
    date: datetime object
        date for which the data is parsed

    Returns
    =======
    dataset: pandas DataFrame object
        transformed option raw data
    '''
    parts = data.split('<table')
    parts2 = parts[1].split('</table')
    dummy = parts2[0].replace(' class="odd"','')
    dummy = dummy.replace(' class="even"','')
    parts3 = dummy.split('<tr><td><b>Total</b>')
    table = parts3[0]   # the html table containing the data
    table = table.replace('class="dataTable"><thead>', 'Pricing day')

    # replace tags by commas and newlines
    table = table.replace('</tr>', '\n')
    table = table.replace(',', '')
    table = table.replace('<td>', ',')
    table = table.replace('</td>', '')
    table = table.replace('<th>', ',')
    table = table.replace('</th>', '')
    table = table.replace('</thead><tbody>', '\n')

    # the resulting string looks like a CSV file
    date_string = date.strftime('%d.%m.%Y')
    table = table.replace('<tr>', date_string)

    string = StringIO(table)  # mask the string as file
    dataset = pd.read_csv(string, parse_dates=[0], index_col=(0, 1),
                dayfirst=True)  # read the 'file' as pandas DataFrame object

    return dataset


def data_collection(path):
    ''' Main function which saves data into the HDF5 file
    'index_option_series.h5' for later use.

    Parameters
    ==========
    path: string
        path to store the data
    '''
    # file to store data
    store = pd.HDFStore(path + 'index_option_series.h5', 'a')

    today = dt.datetime.today()
    start = today - dt.timedelta(31)  # the last 31 days

    day = start.day
    month = start.month
    year = start.year

    for i in range(4):  # iterates over the next 4 months
        dummy_month = month + i
        dummy_year = year
        if dummy_month > 12:
            dummy_month -= 12
            dummy_year += 1

        # collect daily data beginning 31 days ago (start) for
        # option series with expiry dummy_month, dummy_year
        dataset = collect_option_series(dummy_month, dummy_year, start)

        dummy_date = dt.datetime(dummy_year, dummy_month, day)

        # abbreviation for expiry date (for example Oct14)
        series_name = dummy_date.strftime('%b%y')

        if series_name in store.keys():  # if data for that series exists
            index_old = store[series_name].index
            index_new = dataset.index

            if len(index_new - index_old) > 0:
                dummy = pd.concat((store[series_name],
                     dataset.ix[index_new - index_old]))  # add the new data

                store[series_name] = dummy
        else:
            if len(dataset) > 0:
            # if series is new, write whole data set into data store
                store[series_name] = dataset

    store.close()


In [17]:
store = pd.HDFStore('data/index_option_series.h5', 'r')

In [18]:
store


<class 'pandas.io.pytables.HDFStore'>
File path: data/index_option_series.h5

In [19]:
Dec16 = store['Dec16']

In [20]:
Dec16.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1098 entries, (2016-09-23 00:00:00, 1800.0) to (2016-10-20 00:00:00, 3500.0)
Data columns (total 2 columns):
Call_Price    1098 non-null float64
Put_Price     1098 non-null float64
dtypes: float64(2)
memory usage: 20.0 KB


In [21]:
store.close()

In [23]:
Dec16.iloc[25:35]

Call_Price  Put_Price
b' Pricing day' b'Strike price'                       
2016-09-23      2700.0                348.6       29.3
                2725.0                326.8       32.6
                2750.0                305.5       36.2
                2775.0                284.5       40.3
                2800.0                264.0       44.8
                2825.0                243.9       49.7
                2850.0                224.3       55.2
                2875.0                205.3       61.2
                2900.0                186.9       67.8
                2925.0                169.2       75.1

In [26]:
#
# Module with functions to compute VSTOXX sub-indexes
#
# Data as generated by the script index_collect_option_data.py
# is needed for the calculations in this module
#
# (c) Dr. Yves J. Hilpisch
# Listed Volatility and Variance Derivatives
#
import math
import numpy as np
import pandas as pd
import datetime as dt
# import index_date_functions as idf


def compute_subindex(data, delta_T, R):
    ''' Computes a sub-index for given option series data.

    Parameters
    ==========
    data: pandas.DataFrame object
        contains the option data
    delta_T: float
        time interval
    R: float
        discount factor

    Returns
    =======
    subVSTOXX: float
        sub-index value
    '''
    # difference between put and call option with same strike
    data['Diff_Put_Call'] = np.abs(data.Put_Price - data.Call_Price)
    # converts the strike price which serves as index so far
    # to a regular data column
    data = data.reset_index()
    data['delta_K'] = None
    # differences between the different strikes of the series
    data['delta_K'].iloc[1:-1] = [(data['Strike price'][i + 1]
            - data['Strike price'][i - 1]) / 2 for i in data.index[1:-1]]
            # where possible, for the i-th entry it is
            # half of the difference between the (i-1)-th
            # and (i+1)-th price
    #  for i=0 it is just the difference to the next strike
    data['delta_K'].iloc[0] = data['Strike price'][1] - data['Strike price'][0]

    data['delta_K'].iloc[data.index[-1:]] = float(data['Strike price'][-1:]) \
            - float(data['Strike price'][-2:-1])
            # for the last entry, it is just the difference
            # between the second but last strike and the last strike price

    # find the smallest difference between put and call price
    min_index = data.Diff_Put_Call.argmin()

    # the forward price of that option
    forward_price = data['Strike price'][min_index] \
                    + R * data.Diff_Put_Call[min_index]

    K_0 = data['Strike price'][forward_price -
                                    data['Strike price'] > 0].max()
    # the index of the ATM strike
    K_0_index = data.index[data['Strike price'] == K_0][0]

    # selects the OTM options
    data['M'] = pd.concat((data.Put_Price[0:K_0_index],
                           data.Call_Price[K_0_index:]))

    # ATM we take the average of put and call price
    data['M'].iloc[K_0_index] = (data['Call_Price'][K_0_index]
                            + data['Put_Price'][K_0_index]) / 2

    # the single OTM values
    data['MFactor'] = (R * (data['delta_K'] * data['M'])
                         / (data['Strike price']) ** 2)

    # the forward term
    fterm = 1. / delta_T * (forward_price / K_0 - 1) ** 2
    # summing up
    sigma = 2 / delta_T * np.sum(data.MFactor) - fterm
    subVSTOXX = 100 * math.sqrt(sigma)
    return subVSTOXX


def make_subindex(path):
    ''' Depending on the content of the file 'index_option_series.h5',
    the function computes the sub-indexes V6I1, V6I2 and parts
    of V6I3 and returns a pandas DataFrame object with the results.

    Parameters
    ==========
    path: string
        string with path of data file

    Returns
    =======
    df: pandas DataFrame object
        sub-index data as computed by the function
    '''

    # the data source, created with index_collect_option_data.py
    datastore = pd.HDFStore(path + 'index_option_series_2015.h5', 'r')
    
    max_date = dt.datetime.today()  # find the latest date in the source
    for series in datastore.keys():
        dummy_date = datastore[series].index.get_level_values(0)[0]
        dummy_date = dummy_date.to_pydatetime()
        if dummy_date > max_date:
            max_date = dummy_date

    start_date = dt.datetime.today()  # find the earliest date in the source
    for series in datastore.keys():
        dummy_date = datastore[series].index.get_level_values(0)[0]
        dummy_data = dummy_date.to_pydatetime()
        if dummy_date < start_date:
            start_date = dummy_date

    V1 = dict()  # dicts to store the values, V stands for the sub-indices,
                 # T for their expiry
    V2 = dict()
    V3 = dict()
    T1 = dict()
    T2 = dict()
    T3 = dict()

    # from start_date to max_date, but only weekdays
    for day in pd.bdate_range(start=start_date.date(), end=max_date.date()):
        # is V6I1 defined?
        is_V1_defined = not_a_day_before_expiry(day)
        # the settlement date
        settlement_date = first_settlement_day(day)
        # abbreviation for the expiry date, like Oct14
        key = settlement_date.strftime('%b%y')
        # days until maturity
        delta_T = compute_delta(day, settlement_date)
        try:
            # data of the option series for that date
            data = datastore[key].ix[day]
        except:
            continue

        if is_V1_defined:  # if V6I1 is defined
            # compute its value
            V1[day] = compute_subindex(data, delta_T,
                                       math.exp(0.0015 * delta_T))
            T1[day] = settlement_date
        else:
            # compute the value of V6I2 instead
            V2[day] = compute_subindex(data, delta_T,
                                       math.exp(0.0015 * delta_T))
            T2[day] = settlement_date

        settlement_date_2 = idf.second_settlement_day(day)

        # the same for the next index
        key_2 = settlement_date_2.strftime('%b%y')
        delta_T_2 = compute_delta(day, settlement_date_2)
        data_2 = datastore[key_2].ix[day]

        if is_V1_defined:
            V2[day] = compute_subindex(data_2, delta_T_2,
                                       math.exp(0.001 * delta_T_2))
            T2[day] = settlement_date_2
        else:
            V3[day] = compute_subindex(data_2, delta_T_2,
                                       math.exp(0.001 * delta_T_2))
            T3[day] = settlement_date_2

    datastore.close()
    # create the pandas DataFrame object and return it
    df = pd.DataFrame(data={'V6I1': V1, 'Expiry V6I1': T1, 'V6I2': V2,
                    'Expiry V6I2': T2, 'V6I3': V3, 'Expiry V6I3': T3})
    return df



In [29]:
#
# Module to compute VSTOXX index values
# given the values for the relevant sub-indexes
# as generated by the module index_subindex_calculation.py
#
# (c) Dr. Yves J. Hilpisch
# Listed Volatility and Variance Derivatives
#
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def calculate_vstoxx(path):
    ''' Function to calculate the VSTOXX volatility index given time series
    of the relevant sub-indexes.

    Parameters
    ==========
    path: string
        string with path of data files

    Returns
    =======
    data: pandas DataFrame object
        results of index calculation
    '''
    # constant parameters
    seconds_year = 365 * 24 * 3600.
    seconds_30_days = 30 * 24 * 3600.

    # import historical VSTOXX data
    data = pd.read_csv(path + 'vs.csv', index_col=0, parse_dates=True)

    # determine the settlement dates for the two underlying option series
    data['Settlement date 1'] = [first_settlement_day(a) for a in data.index]
    data['Settlement date 2'] = [second_settlement_day(a) for a in data.index]

    # deduce the life time (in seconds) from current date to
    # final settlement Date
    data['Life time 1'] = [(data['Settlement date 1'][i] - i).days
                            * 24 * 60 * 60 for i in data.index]
    data['Life time 2'] = [(data['Settlement date 2'][i] - i).days
                            * 24 * 60 * 60 for i in data.index]

    data['Use V6I2'] = data['V6I1'].notnull()  # where V6I1 is not defined
    data['Subindex to use 1'] = [data['V6I1'][i] if data['Use V6I2'][i]
                        else data['V6I2'][i] for i in data.index]
                        # if V6I1 is defined, use V6I1 and V6I2 as data set
    data['Subindex to use 2'] = [data['V6I2'][i] if data['Use V6I2'][i]
                        else data['V6I3'][i] for i in data.index]
                        # else use V6I2 and V6I3

    # the linear interpolation of the VSTOXX value
    # from the two relevant sub-indexes
    data['Part 1'] = data['Life time 1'] / seconds_year \
                        * data['Subindex to use 1'] ** 2 \
                        * ((data['Life time 2'] - seconds_30_days)
                        / (data['Life time 2'] - data['Life time 1']))

    data['Part 2'] = data['Life time 2'] / seconds_year \
                        * data['Subindex to use 2'] ** 2 \
                        *((seconds_30_days - data['Life time 1'])
                        / (data['Life time 2'] - data['Life time 1'])) 

    data['VSTOXX'] = np.sqrt((data['Part 1'] + data['Part 2']) *
                        seconds_year / seconds_30_days)

    # difference between original VSTOXX data and re-calculated values
    data['Difference'] = data['V2TX'] - data['VSTOXX']

    return data
